## Estimated displaced travel and GHG emissions

This notebook takes the generated quantities from the preferred model, and uses them to calculate the km displaced and GHG impacts of different scenarios for bike lane construction.

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import math

from shapely import wkt
from sklearn import preprocessing

In [ ]:
# run this twice, with and without transit in displaced km
exclude_transit = False
fn_suffix='_excltransit' if exclude_transit else ''  # save files under different name
model_type='beta' #'linear'

# where the estimates are saved in Notebook 2 (stanmodel)
outputpath = '../stan_output/'

In [ ]:
city_data=pd.read_csv('data/data_23.csv', index_col=0).reset_index()
city_data['geometry'] = city_data.centroid.apply(wkt.loads)

# load in aggregate estimates
agg_medians = pd.read_csv(outputpath+'gq_'+model_type+'_medians_aggregated'+fn_suffix+'.csv').set_index('mode')
agg_pc_025 = pd.read_csv(outputpath+'gq_'+model_type+'_pc_025_aggregated'+fn_suffix+'.csv').set_index('mode')
agg_pc_975 = pd.read_csv(outputpath+'gq_'+model_type+'_pc_975_aggregated'+fn_suffix+'.csv').set_index('mode')

for adf in [agg_medians, agg_pc_025, agg_pc_975]:
    adf['pc_reduction'] = adf['pc_reduction']*100    

# add in values for zero bike lanes, which aren't modeled but effects are always zero
zerodf = pd.DataFrame(np.zeros((3,6)), columns=agg_medians.columns, index=['_combined','_walk','_bike',])
agg_medians = pd.concat([zerodf, agg_medians])
agg_pc_025 = pd.concat([zerodf, agg_pc_025])
agg_pc_975 = pd.concat([zerodf, agg_pc_975])

In [ ]:
# calculate cumulative km of bike lanes that would be added
xmax = 50 # limit to how many bike lanes we want to show

new_bl_mkm = []
for bl in range(xmax+1):
    km = np.maximum(0, city_data['roads_km'] * bl/100 - city_data['bikelane_length_km']).sum()
    new_bl_mkm.append(km/1e6)
new_bl_mkm = pd.Series(new_bl_mkm, name='new_bikelane_km')

In [ ]:
# calculate impacts under alternatives for displacement ratio
# this is a slight approximation because the combined is always a little more than walk and bike combined
# due to how negative numbers are ignored

cph  = round(city_data.loc[city_data.city=='Copenhagen', 'bikelane_per_road_km'].values[0] *100)
pc95 = round(city_data.bikelane_per_road_km.quantile(q=0.95)*100)
agg_medians[agg_medians.bikelanes==cph]

bike_displacements = [1.0, 0.8, 2.1, 0.9]
walk_displacements = [1.0, 1.0, 1.0, 3.1]

for bd, wd in zip(bike_displacements, walk_displacements):
    for scenario, sname in zip([cph, pc95],['Copenhagen','95th percentile']):
        bpc = agg_medians[agg_medians.bikelanes==scenario].loc['_bike','pc_reduction'] * bd
        wpc = agg_medians[agg_medians.bikelanes==scenario].loc['_walk','pc_reduction'] * wd
        print(f'Scenario {sname} with displacements {bd} and {wd}: {np.round(bpc+wpc,2)}')
    


In [ ]:
colors = ['#1b9e77','#d95f02','#7570b3'] # colorbrewer2, 3 colors, colorblind safe
labels = ['Walking+bicycling','Walking', 'Bicycling']
modes = ['_combined','_walk','_bike',]

# seaborn warning
warnings.simplefilter(action='ignore', category=FutureWarning)

fig, ax = plt.subplots(figsize=(7.5, 5))

for mode, c, l in zip(modes, colors, labels):
    sns.lineplot(data=agg_medians.loc[agg_medians.bikelanes<=xmax].loc[mode], x='bikelanes', y='pc_reduction', label = l, color=c, ax=ax)
    if mode=='_combined': # add CI
        sns.lineplot(data=agg_pc_025.loc[agg_pc_025.bikelanes<=xmax].loc[mode], x='bikelanes', y='pc_reduction', color=c, alpha=0.2, ax=ax)
        sns.lineplot(data=agg_pc_975.loc[agg_pc_975.bikelanes<=xmax].loc[mode], x='bikelanes', y='pc_reduction', color=c, alpha=0.2, ax=ax)
        line = ax.get_lines()[-3:]
        plt.fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), color=c, alpha=.1)
   
if 0: # add cumulative bike lane line
    new_bl_mkm.plot(color='k', lw=1.5, ax=ax, secondary_y=True)
    # fake plot for legend
    sns.lineplot(x=(0,0), y=(0,0), ax=ax, label = 'Extent of new bicycle lanes', lw=1.5, color='k')
    ax.right_ax.set_ylabel('Length of new bicycle lanes (million km)', size=12)

ax.get_legend().remove()
ax.text(32, 3.80, 'Walking+\nbicycling', color=colors[0], ha='center', weight='bold', size=12)
ax.text(40, 3.25, 'Walking', color=colors[1], ha='center', weight='bold', size=12)
ax.text(40, 0.85, 'Bicycling', color=colors[2], ha='center', weight='bold', size=12)

ax.set_ylim(0)
ax.set_xlim(0,xmax)
ax.set_xlabel('Bicycle facility provision (km per 100km of road)', size=12)
ax.set_ylabel('Percent reduction in private vehicle emissions', size=12)
ax.set_yticks(range(0,13,2,))

# add vertical lines and labels
cph = (city_data.loc[city_data.city=='Copenhagen','bikelane_per_road_km']*100).values
meancity = city_data.bikelane_per_road_km.mean()*100
mediancity = city_data.bikelane_per_road_km.median()*100
topcity = city_data.bikelane_per_road_km.quantile(q=0.95)*100

for xpos, xname in zip([cph, meancity, mediancity, topcity], 
                       ['Copenhagen', 'City-level mean', 'City-level median', '95th percentile city']):
    ax.axvline(xpos, color='0.5', linestyle='--', linewidth=1)
    ax.text(xpos+0.5, 7.5, xname, va='top', rotation=90)

plt.tight_layout()
fig.savefig(outputpath+'displacement_'+model_type+'_'+fn_suffix+'.png', dpi=1200)
fig.savefig(outputpath+'displacement_'+model_type+'_'+fn_suffix+'.pdf')


In [ ]:
# report some metrics
f = open(outputpath+'displacement_stats_'+model_type+'_'+fn_suffix+'.txt', 'w')

def printwrite(line, outfile=f):
    print(line)
    outfile.write(str(line)+'\n')

if exclude_transit:
    printwrite('Estimates assume displacement is only cars/mc')
else:
    printwrite('Estimates assume displacement is to cars/mc/transit')
    
dollarvalues = {'_combined': 0, '_walk':0.79, '_bike': 0.50} # health benefits

walk_co2 = agg_medians[agg_medians.bikelanes<=50].loc['_walk','pc_reduction'].reset_index(drop=True)
bike_co2 = agg_medians[agg_medians.bikelanes<=50].loc['_bike','pc_reduction'].reset_index(drop=True)

printwrite('Current length of bike lanes in all cities (million km): {:.1f}\n'.format(city_data.bikelane_length_km.sum()/1e6), f)
printwrite('Bike lanes per 100km road in median city: {:.1f}\n'.format(city_data.bikelane_per_road_km.median()*100), f)
printwrite('Total pop in all cities (bn): {}'.format(city_data.population.sum()/1e9), f)

bls = {'mean': city_data.bikelane_per_road_km.mean()*100,
       '95pc': city_data.bikelane_per_road_km.quantile(q=0.95)*100,
       'cph': city_data.loc[city_data.city=='Copenhagen', 'bikelane_per_road_km'].values[0] *100}

for i in bls:
    printwrite('\nImpact of increasing to {} city:'.format(i))
    # figure out midpoint between integer values that are modeled
    bl_bounds = new_bl_mkm.loc[math.floor(bls[i])], new_bl_mkm.loc[math.ceil(bls[i])]
    printwrite('Adds {:.1f} million km of new bike lanes across all cities ({:.1f} km per 100km road)'.format((bl_bounds[1]-bl_bounds[0]) * (bls[i]%1) + bl_bounds[0], bls[i]))
    for dv in ['pc_reduction','co2_kg','added_km']:
        divider = 1 if dv =='pc_reduction' else 1e9
        units = '%' if dv =='pc_reduction' else 'bn'
        hb = np.array([0,0,0])
        for mode in ['_combined','_walk','_bike']:  
            if dv!='added_km' and mode!='_combined': continue
            r = []
            for adf in [agg_medians, agg_pc_025, agg_pc_975]:
                bl_bounds = adf[adf.bikelanes==math.floor(bls[i])].loc[mode,dv], adf[adf.bikelanes==math.ceil(bls[i])].loc[mode,dv]  
                r.append(((bl_bounds[1]-bl_bounds[0]) * (bls[i]%1) + bl_bounds[0])/divider)
            printwrite('{} for {}: {:.1f}{} (95% UI: {:.2f} to {:.2f})'.format(dv, mode[1:], r[0], units, r[1], r[2]))
            if dv=='added_km' and mode!='_combined': # dollar value of extra km health benefits
                 hb = hb + np.array(r) * dollarvalues[mode]
                 if mode=='_bike': 
                    printwrite('  Health benefits: US${:.2f}{} (95% UI: {:.2f} to {:.2f})'.format(hb[0], units, hb[1], hb[2]))

    mask = agg_medians.bikelanes==math.floor(bls[i])
    pc_from_walk  = agg_medians[mask].loc['_walk','co2_kg'] / (agg_medians[mask].loc['_walk','co2_kg'] + agg_medians[mask].loc['_bike','co2_kg'])
    print('{:.2f}% of the CO2 reductions are from walking'.format(pc_from_walk*100))

# What is our implied CO2 savings per new walk or bike trip?
printwrite('\nCO2 savings')
for mode in ['walk', 'bike']:
    co2_per_trip = (agg_medians.loc['_'+mode,'co2_kg'] / agg_medians.loc['_'+mode,'added_km']).mean()
    printwrite('{}: {:.2f} per km'.format(mode, co2_per_trip))
    if mode=='walk':
        triplength = city_data.km_on_foot.sum()/city_data.trips_on_foot_touse.sum()
    else:
        triplength = city_data.km_cycling.sum()/city_data.trips_cycling_touse.sum()
    printwrite('{}: {:.2f} per trip'.format(mode, co2_per_trip*triplength))

print('\nCities at 95th percentile:') # 11007.65
i, j = math.floor(len(city_data)*.95), math.ceil(len(city_data)*.95)
print(city_data.sort_values(by='bikelane_per_road_km').iloc[i-1:j+1][['city','country','bikelane_per_road_km']])

f.close()
